## Problem Definition ##
In this example, we’ll be training a neural network using particle swarm optimization. For this we’ll be using the standard global-best PSO pyswarms.single.GBestPSO for optimizing the network’s weights and biases. This aims to demonstrate how the API is capable of handling custom-defined functions.

For this example, we’ll try to classify the three iris species in the Iris Dataset using 2 layer neural network.

Reference: https://pyswarms.readthedocs.io/en/latest/examples/usecases/train_neural_network.html

In [1]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
# Import PySwarms
import pyswarms as ps

In [2]:
# Load IRIS Dataset
data = load_iris()

In [3]:
# Store the features as X and the labels as y
X = data.data
y = data.target

n_inputs = 4
n_hidden = 20
n_classes = 3

num_samples = 150

In [4]:
def logits_function(p):
    """ Calculate roll-back the weights and biases

    Inputs
    ------
    p: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    numpy.ndarray of logits for layer 2

    """
    # Roll-back the weights and biases
    W1 = p[0:80].reshape((n_inputs,n_hidden))
    b1 = p[80:100].reshape((n_hidden,))
    W2 = p[100:160].reshape((n_hidden,n_classes))
    b2 = p[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2



In [5]:
# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """

    logits = logits_function(params)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss


In [6]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)


In [7]:
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

2020-11-17 04:01:16,241 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.132
2020-11-17 04:01:24,089 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.13217636571868263, best pos: [-0.08513914  0.91234116  0.57854326 -0.84570455 -0.11775975  1.29935242
  0.75900161  0.12136719  0.26145413  0.23115017 -0.13817435 -0.02190398
  0.33678135  0.16316093  1.56425064 -0.95945268 -0.1519693  -0.85570436
  1.16803532  0.67823521 -0.42384241 -1.03678939  0.38133575  0.00915552
  0.13385548 -1.00776324 -0.34169834 -0.08873128  0.14295963 -1.16205589
 -0.68044005 -0.73809227  0.40256158  0.59066308 -1.35409668  0.61889887
 -0.47562718 -0.61526335  0.17787205  0.23886885 -0.06423171  0.68219108
 -1.6286412  -0.21292232  0.15898432 -0.70866008 -0.42103409  0.02484263
  0.04338663  0.88272492  0.53929408 -0.62684297  0.38161909 -0.41859346
  0.49479626  1.0